# Diffusion Convolutional Neural Network
原文地址：1707.01926v3<br/>
代码地址：https://github.com/liyaguang/DCRNN<br/>
万物皆可推广. 本质上，是共享权重的多层图卷积网络<br/>
扩散的步数越多，在空间维度数据越平滑. 过犹不及，原文中只是将GCN中的一步扩散推广成两步. <br/>
![avatar](img/dcrnn.PNG)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Layer, Dense

import numpy as np
import scipy.sparse as sp
import matplotlib as plot

In [ ]:
# * 探索代码
'''
伪代码阶段
调整维度
调通代码
'''

# Code Repo

In [ ]:
class GCN(Layer):
    def __init__(self):
        super(self, GCN).__init__()
    def build(self, diffusion_matrix, dim_input, dim_hidden, k = 2):
        self.diffusion_matrix = diffusion_matrix
        self.w = tf.Variable(shape=(dim_input, dim_hidden))
        self.k = k
    def call(self, inputs):
        # 计算扩散卷积.
        y = []
        y_t_minus_2 = inputs
        y.concat(y_t_minus_2) # y_0 = x
        y_t_minus_1 = self.diffusion_matrix @ y_t_minus_2 # fence-post problem, y_1 = Dx
        y.concat(y_t_minus_1)
        for _k in range(0, self.k):
            y_t = 2 * self.diffusion_matrix @ y_t_minus_1 - y_t_minus_2
            y.concat(y_t) # y_t = 2 * Dy_t-1 - yt-2，既是切比雪夫递归多项式(ChebNet)，又是扩散方程(热传导方程)的渐进解.
        return y * self.w

In [ ]:
'''
Input
  A: adjacency matrix
'''
def calculate_normalized_laplacian(adj):
    adj = sp.coo_matrix(adj)
    d = np.array(adj.sum(1))
    d_inv_sqrt = np.power(d, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    normalized_laplacian = sp.eye(adj.shape[0]) - adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    return normalized_laplacian